# Single Run

In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import json

from datetime import datetime, date


from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor

from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BudgetSetImplementor



np.random.seed(seed = 10)


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_56798/3106561885.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
num_implementor_adversary_iterations_max = 15

In [3]:
task_names = ['trials_task_100_1',
              'trials_task_100_2',
              'trials_task_100_3',
              'trials_task_200_1',
              'trials_task_200_2',
              'trials_task_200_3'
              ]


def extract_version(filename):
    match = re.search(r'task_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        raise ValueError("The pickle name do not match the structure")
        
        
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (datetime, date)):
            return obj.isoformat()
        elif isinstance(obj, tuple):
            return str(obj)  # Converti le tuple in stringhe
        return super(CustomEncoder, self).default(obj)     
        
def dump_dict_to_json(dictionary, filename):
    with open('results/' + filename + '.json', 'w') as json_file:
        json.dump(dictionary, json_file, cls=CustomEncoder, indent=4)

def to_pandas_to_csv(data, filename):
    data = pd.DataFrame(data)
    data.to_csv('results/' + filename + '.csv')
    
    
def export_all_artifacts(schedule, report, task_file_name):
    dump_dict_to_json(dictionary=schedule,
                      filename=task_file_name + '_schedule')
    
    dump_dict_to_json(dictionary=report,
                      filename=task_file_name + '_report')
    
    to_pandas_to_csv(data = schedule.get('patients_not_included'),
                     filename = task_file_name + '_patients_not_included')
    
    to_pandas_to_csv(data = schedule.get('blocks'),
                     filename = task_file_name + '_schedule_blocks')

In [4]:
for task_name in task_names:
    
    num_of_patients, version = extract_version(task_name)
    
    print("Task:", task_name, "pats:", num_of_patients, "version", version)
       
    task_file = open('task/' + task_name + '.pickle', 'rb')
    task = pickle.load(task_file)                    
    task_file.close()
    
    # Budget set Gamma Settings
     
    task.gamma_max = 10
    task.gamma_variation = 0

    
    
    ### FIRST ROBUSTNESS PROFILE
    
    print("Robustness profile 1:", task.robustness_risk, task.robustness_overtime)
    
    ##
    
    print("\tImplementory Adversary")
    
    implementor_adversary = ImplementorAdversary(task = task, 
                                             implementor=StandardImplementor(), 
                                             adversary=None, 
                                             description = 'Trials')
    implementor_adversary.create_instance()
    
    schedule, report, _ = implementor_adversary.run(num_implementor_adversary_iterations_max)
    
    export_all_artifacts(schedule=schedule, 
                         report=report, 
                         task_file_name=task_name + "_impladv_rb1_")
    
    ##
    
    print("\tChance Constraints")
    
    chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Trials')
    chance_constraints.create_instance()
    
    schedule, report = chance_constraints.run()
    
    export_all_artifacts(schedule=schedule, 
                         report=report, 
                         task_file_name=task_name + "_chancec_rb1_")    
    
    ##
    
    print("\tBudget Set")
    
    budget_set = BudgetSet(task = task, 
                                implementor=BudgetSetImplementor(task = task), 
                                description = 'Trials')

    schedule, report = budget_set.run()
    
    export_all_artifacts(schedule=schedule, 
                        report=report, 
                        task_file_name=task_name + "_budgets_rb1_")  
    
    
    
    ### SECOND ROBUSTNESS PROFILE
    
    task.set_robustness_risk = 0.1
    task.set_robustness_overtime= 10
    
    print("Robustness profile 2:", task.robustness_risk, task.robustness_overtime)


    print("\tImplementory Adversary")
    
    implementor_adversary = ImplementorAdversary(task = task, 
                                             implementor=StandardImplementor(), 
                                             adversary=None, 
                                             description = 'Trials')
    implementor_adversary.create_instance()
    
    schedule, report, _ = implementor_adversary.run(num_implementor_adversary_iterations_max)
    
    export_all_artifacts(schedule=schedule, 
                         report=report, 
                         task_file_name=task_name + "_impladv_rb2_")
    
    ##
    
    print("\tChance Constraints")
    
    chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Trials')
    chance_constraints.create_instance()
    
    schedule, report = chance_constraints.run()
    
    export_all_artifacts(schedule=schedule, 
                         report=report, 
                         task_file_name=task_name + "_chancec_rb2_")    
    
    ##
    
    print("\tBudget Set")

    budget_set = BudgetSet(task = task, 
                                implementor=BudgetSetImplementor(task = task), 
                                description = 'Trials')

    schedule, report = budget_set.run()
    
    export_all_artifacts(schedule=schedule, 
                        report=report, 
                        task_file_name=task_name + "_budgets_rb2_")  
    
    
    
    
    print("Trials completed!")
    

Task: trials_task_100_1 pats: 100 version 1
Robustness profile 1: 0.2 30
	Implementory Adversary


In [ ]:
schedule

In [ ]:
def export_all_artifacts(schedule, report, task_file_name):
    dump_dict_to_json(dictionary=schedule,
                      filename=task_file_name + '_schedule')
    
    dump_dict_to_json(dictionary=report,
                      filename=task_file_name + '_report')
    
    to_pandas_to_csv(data = schedule.get('patients_not_included'),
                     filename = task_file_name + '_patients_not_included')
    
    to_pandas_to_csv(data = schedule.get('blocks'),
                     filename = task_file_name + '_schedule_blocks')